<a href="https://colab.research.google.com/github/dabidstudio/python-example-scripts/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84%EB%89%B4%EC%8A%A4_%EB%B9%84%EC%8A%B7%ED%95%9C%EA%B8%B0%EC%82%AC%EC%84%A0%EB%B3%84_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버뉴스 유사한 기사를 임베딩(embedding) 이용하여 선별하기



### 지금 우리 상황은? 중복된 기사를 선별해야 함
- 네이버 뉴스목록을 엑셀파일 형태로 정리하였음
- 그런데 중복되는 기사들이 많음
- 텍스트가 정확하게 일치하는건 아닌데, 비슷한 단어와 구절이 있어서 사람이 봐서는 구별할 수 있음
- 그런데 전체 기사가 너무 많아서 사람이 일일이 선별하는건 번거로움
- "어떻게 하면 중복되는 기사는 선별해서, 주제나 내용별로 대표 기사 1개만 선별할 수 있을까?"


![](https://drive.google.com/uc?export=view&id=1zAEqoCNy_D-4lBrZaUYMWuVJhNN-BF82)


이런 상황에서 임베딩(Embedding)를 이용하면 문제를 효과적으로 해결할 수 있음



### 임베딩(Embedding) 이란?
- 텍스트를 컴퓨터가 처리할 수 있도록 숫자들의 나열로 바꾸는 작업
- 원래는 이 작업을 위해서는 복잡한 알고리즘으로 구성된 AI 모델을 직접 훈련시켜서 활용했어야 함
- 다행히도 Open AI가 잘 훈련된 모델을 API 형태로 제공해주고 있어서, 이를 활용하면 쉽게 텍스트 덩어리 → 숫자 나열로 변환 가능 (https://platform.openai.com/docs/api-reference/embeddings/create)
- 모델마다 반환하는 숫자 형태가 다른데, Open AI의 모델은 1,536개의 숫자를 리스트로 반환 (벡터)
- 조금 다르게 표현하면, 1,536차원에서 존재하는 하나의 점 형태로 반환
![how-it-works](https://cdn.openai.com/embeddings/draft-20220124e/vectors-1.svg)


### 유사한 기사제목을 Embeddings로 구하는 로직

- 유사한 텍스트의 정의를 하는 것이 중요
  - 정의 1 : 점의 거리가 가까울수록 유사하다! : 유클리드 거리 (Euclidean Distance)
  - 정의 2 : 2개 점이 만들어내는 각도가 작을수록 유사하다! : 코사인 유사도(cosine similarity)
    - 각도가 0° = 완전 유사(정확하다) = 코사인 유사도 (cosine 0°의 값)= 1
    - 각도가 90° = 전혀 관련성이 없다 = 코사인 유사도 = 0

![](https://drive.google.com/uc?export=view&id=1ClkO42wFldiTwu2dCwLanXXG_q17lZCq)
  

- 1,536차원에서 2개의 점의 거리가 가까울수록 유사하다! 라고 정의
- 기사제목 1개가 점 1개라고 가정하는 경우 → 다른 기사제목이 이 기사랑 얼마나 가까운지 거리를 계산하고 비교하면 가능!
- 예를 들어10개의 기사제목이 있으면, 서로 모든 경우에 대해서 거리를 계산해봐야 함 (10 * 9 / 2 = 45)
- 코드를 이용해서 이 계산을 하고, 특정 기준 이상으로 가까우면 같은 그룹으로 배정!




## 실습

### 라이브러리 불러오기

In [ ]:

import pandas as pd
import requests
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

### 핵심 상수·함수 선언

In [ ]:
## openai apikey 입력
OPENAI_API_KEY = "" #@param {type:""}


In [ ]:



## 1) 텍스트가 있으면 임베딩 (벡터)를 반환해주는 함수

def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ' + OPENAI_API_KEY
    }
    json_data = {
        'input': text,
        'model': model,
    }
    try:
        response = requests.post('https://api.openai.com/v1/embeddings', headers=headers, json=json_data)
        if response.status_code == 200:
            embedding = response.json()['data'][0]['embedding']
            return embedding
        else:
            print(f"임베딩 가져오기 오류: HTTP {response.status_code} - {response.text}")
            return None
    except Exception as e:
        # 다른 예외 처리
        print(f"임베딩 가져오기 오류: {e}")
        return None

In [ ]:
## 필요한 파일 업로드 후 dataframe으로 변환

import io
from google.colab import files
uploaded = files.upload()
file_path = list(uploaded.keys())[0]


Saving 샘플데이터.xlsx to 샘플데이터 (4).xlsx


In [ ]:
df = pd.read_excel(file_path)
df

,제목,원문링크,네이버뉴스링크,상세내용,발행일자
0,"뉴타운급 재개발·재건축, 수요자 관심 높아져",https://www.dnews.co.kr/uhtml/view.jsp?idxno=2...,https://www.dnews.co.kr/uhtml/view.jsp?idxno=2...,"단지는 지하 3층~지상 최고 39층, 9개 동, 총 1,370 세대 규모로 이 중 ...",2024-02-09 15:22:52
1,진돗개는 다 사납다? 이 글 읽으면 생각 달라질 걸요,https://www.ohmynews.com/NWS_Web/View/at_pg.as...,https://n.news.naver.com/mnews/article/047/000...,88 <b>서울</b>올림픽 개막식 때 진행된 '진돗개 퍼레이드'도 여기 기름을 부...,2024-02-09 14:46:52
2,"GS건설, '영통자이 센트럴파크' 견본주택 15일 개관",http://www.econonews.co.kr/news/articleView.ht...,http://www.econonews.co.kr/news/articleView.ht...,GS건설은 수원시 영통구 영통동에서 '영통자이 센트럴파크' 견본주택을 오는 15일 ...,2024-02-09 14:24:52
3,가라앉은 분양시장…설 이후 살아날까,https://www.newsway.co.kr/news/view?ud=2024020...,https://www.newsway.co.kr/news/view?ud=2024020...,"이어 인천 5194가구, <b>서울</b> 1236가구(567가구) 등의 순이다. ...",2024-02-09 14:22:52
4,경기도 성남시 '분당 금호어울림 그린파크' 242세대 공공분양…입주자 모집,http://www.econonews.co.kr/news/articleView.ht...,http://www.econonews.co.kr/news/articleView.ht...,경기도 성남시가 야탑동 공공<b>분양</b>주택 건립사업인 '분당 금호어울림 그린파...,2024-02-09 14:22:52
...,...,...,...,...,...
82,"유기견 8마리 입양해 굶기고 때리고 잔혹하게 살해한 20대, ‘법정최고형’ 징...",http://www.segye.com/newsView/20240208512490?O...,https://n.news.naver.com/mnews/article/022/000...,"8일 법조계에 따르면, 전날 <b>서울</b>고법 춘천재판부 형사1부(김형진 부장판...",2024-02-08 17:17:52
83,"한달 새 3천건 늘어난 수도권 미분양…시장에선 ""정부, 미분양 특화 대책 내놔...",http://www.goodkyung.com/news/articleView.html...,http://www.goodkyung.com/news/articleView.html...,실제로 민간 아파트 평당 <b>분양</b>가격은 <b>서울</b>은 1년 새 17%...,2024-02-08 17:16:52
84,내년 공동주택 입주예정물량 27.5만가구…올해보다 24%↓,https://www.yna.co.kr/view/AKR2024020815540000...,https://n.news.naver.com/mnews/article/001/001...,내년 입주예정 물량을 지역별로 보면 <b>서울</b>은 3만1천265가구로 올해(2...,2024-02-08 17:13:52
85,강남3구도 고꾸라졌다… 집값 반년만에 13억 빠져,http://www.fnnews.com/news/202402081701066947,https://n.news.naver.com/mnews/article/014/000...,<b>서울</b> 강남3구(강남·서초·송파)의 아파트 하락폭이 깊어지고 있다. 강남...,2024-02-08 17:02:52


In [ ]:
## 샘플로 1개만 embedding 돌려보기

sample_text = df.loc[0,'제목']
sample_text

'뉴타운급 재개발·재건축, 수요자 관심 높아져'

In [ ]:
embedding = get_embedding(sample_text)
embedding

[-0.022135263,
 0.0064464305,
 0.058267828,
 0.041619875,
 -0.021205211,
 0.0033016885,
 -0.00037892399,
 0.037202124,
 -0.024134878,
 -0.041759383,
 0.023262953,
 0.005554161,
 0.018356923,
 0.013253257,
 0.0490603,
 -0.029110663,
 -0.026669273,
 -0.08193768,
 -0.026599519,
 -0.018763822,
 0.026599519,
 -0.018124409,
 0.01436932,
 -0.025367199,
 0.023669852,
 0.04220116,
 0.006853329,
 -0.0075159916,
 -0.023716355,
 -0.0031273037,
 -0.0034353836,
 -0.03659759,
 -0.008039147,
 0.021484226,
 0.009922504,
 0.03466773,
 0.028343368,
 -0.02182137,
 -0.010439846,
 0.020147275,
 -0.0028817113,
 -0.008730873,
 -0.0062022917,
 -0.009219151,
 0.005882586,
 0.019472986,
 -0.041736133,
 0.034946747,
 -0.013602027,
 0.048037242,
 0.021926003,
 -0.012916112,
 0.03538852,
 -0.02107733,
 0.024297638,
 0.036458082,
 0.0075915586,
 -0.03306339,
 0.026878534,
 -0.02005427,
 -0.00035658092,
 -0.040852584,
 0.010637483,
 0.011056006,
 -0.012392958,
 -0.027390065,
 0.023344332,
 0.016450314,
 -0.03480724,


### 모든 행의 제목에 대해서 embedding을 구하고, 열 형태로 저장


In [ ]:
# 임베딩을 저장할 빈 리스트 초기화
embeddings = []

# DataFrame의 각 행에 대해 반복
for index, row in df.iterrows():
    text = row["제목"]
    ## 벡터로 제대로 인식할 수 있도록 np.array() 이용,
    embedding = np.array(get_embedding(text))
    embeddings.append(embedding)

df['np_embedding'] = embeddings

In [ ]:
df

,제목,원문링크,네이버뉴스링크,상세내용,발행일자,np_embedding
0,"뉴타운급 재개발·재건축, 수요자 관심 높아져",https://www.dnews.co.kr/uhtml/view.jsp?idxno=2...,https://www.dnews.co.kr/uhtml/view.jsp?idxno=2...,"단지는 지하 3층~지상 최고 39층, 9개 동, 총 1,370 세대 규모로 이 중 ...",2024-02-09 15:22:52,"[-0.022135263, 0.0064464305, 0.058267828, 0.04..."
1,진돗개는 다 사납다? 이 글 읽으면 생각 달라질 걸요,https://www.ohmynews.com/NWS_Web/View/at_pg.as...,https://n.news.naver.com/mnews/article/047/000...,88 <b>서울</b>올림픽 개막식 때 진행된 '진돗개 퍼레이드'도 여기 기름을 부...,2024-02-09 14:46:52,"[0.048575576, -0.002206344, 0.0013420887, 0.02..."
2,"GS건설, '영통자이 센트럴파크' 견본주택 15일 개관",http://www.econonews.co.kr/news/articleView.ht...,http://www.econonews.co.kr/news/articleView.ht...,GS건설은 수원시 영통구 영통동에서 '영통자이 센트럴파크' 견본주택을 오는 15일 ...,2024-02-09 14:24:52,"[-0.012990458, 0.039917395, 0.039548215, 0.009..."
3,가라앉은 분양시장…설 이후 살아날까,https://www.newsway.co.kr/news/view?ud=2024020...,https://www.newsway.co.kr/news/view?ud=2024020...,"이어 인천 5194가구, <b>서울</b> 1236가구(567가구) 등의 순이다. ...",2024-02-09 14:22:52,"[0.046844117, -0.0089707635, 0.024460474, 0.02..."
4,경기도 성남시 '분당 금호어울림 그린파크' 242세대 공공분양…입주자 모집,http://www.econonews.co.kr/news/articleView.ht...,http://www.econonews.co.kr/news/articleView.ht...,경기도 성남시가 야탑동 공공<b>분양</b>주택 건립사업인 '분당 금호어울림 그린파...,2024-02-09 14:22:52,"[0.012976763, 0.004866286, 0.039385192, -0.020..."
...,...,...,...,...,...,...
82,"유기견 8마리 입양해 굶기고 때리고 잔혹하게 살해한 20대, ‘법정최고형’ 징...",http://www.segye.com/newsView/20240208512490?O...,https://n.news.naver.com/mnews/article/022/000...,"8일 법조계에 따르면, 전날 <b>서울</b>고법 춘천재판부 형사1부(김형진 부장판...",2024-02-08 17:17:52,"[0.05710743, 0.087503955, 0.00020387991, 0.009..."
83,"한달 새 3천건 늘어난 수도권 미분양…시장에선 ""정부, 미분양 특화 대책 내놔...",http://www.goodkyung.com/news/articleView.html...,http://www.goodkyung.com/news/articleView.html...,실제로 민간 아파트 평당 <b>분양</b>가격은 <b>서울</b>은 1년 새 17%...,2024-02-08 17:16:52,"[0.021479802, 0.010624549, 0.026761722, 0.0161..."
84,내년 공동주택 입주예정물량 27.5만가구…올해보다 24%↓,https://www.yna.co.kr/view/AKR2024020815540000...,https://n.news.naver.com/mnews/article/001/001...,내년 입주예정 물량을 지역별로 보면 <b>서울</b>은 3만1천265가구로 올해(2...,2024-02-08 17:13:52,"[0.02662226, 0.04503617, -0.009569089, 0.03300..."
85,강남3구도 고꾸라졌다… 집값 반년만에 13억 빠져,http://www.fnnews.com/news/202402081701066947,https://n.news.naver.com/mnews/article/014/000...,<b>서울</b> 강남3구(강남·서초·송파)의 아파트 하락폭이 깊어지고 있다. 강남...,2024-02-08 17:02:52,"[0.002876054, 0.03164961, 0.059259728, -0.0280..."


### 코사인 유사도를 구하는 간단한 샘플 먼저 보기


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 한국어 텍스트 설명 및 임베딩이 포함된 샘플 데이터
data = {
    'text': ['아이템 1에 대한 텍스트', '아이템 2에 대한 텍스트', '아이템 3에 대한 텍스트'],
    'np_embedding': [np.array([1, 2]), np.array([2, 2]), np.array([3, 4])]
}

# 데이터프레임 생성
sample_df = pd.DataFrame(data)
# 텍스트와 해당 임베딩이 포함된 데이터프레임 표시
print("텍스트와 임베딩이 포함된 데이터프레임:\n", sample_df, "\n")  # "텍스트와 임베딩이 포함된 데이터프레임"

# 2D 배열 형태로 임베딩 쌓기
stacked_embeddings = np.stack(sample_df['np_embedding'].values)
# 쌓인 임베딩과 유사도 행렬 출력
print("스택된 임베딩:\n", stacked_embeddings)  # "스택된 임베딩"



# 코사인 유사도 행렬 계산
similarity_matrix = cosine_similarity(stacked_embeddings)

# 유사도 행렬을 DataFrame으로 변환하여 더 명확하게 표시
similarity_df = pd.DataFrame(similarity_matrix, columns=['아이템 1', '아이템 2', '아이템 3'], index=['아이템 1', '아이템 2', '아이템 3'])
print("코사인 유사도 행렬:\n", similarity_df, "\n")

# 특정 셀의 유사도 값을 직접 계산하여 설명
item1_embedding = sample_df.loc[0, 'np_embedding']
item2_embedding = sample_df.loc[1, 'np_embedding']

# 직접 코사인 유사도 계산
cosine_sim = np.dot(item1_embedding, item2_embedding) / (np.linalg.norm(item1_embedding) * np.linalg.norm(item2_embedding))

print("아이템 1과 아이템 2 사이의 코사인 유사도:", cosine_sim)
print("유사도 행렬에서 아이템 1과 아이템 2 사이의 값:", similarity_df.loc['아이템 1', '아이템 2'])


텍스트와 임베딩이 포함된 데이터프레임:
             text np_embedding
0  아이템 1에 대한 텍스트       [1, 2]
1  아이템 2에 대한 텍스트       [2, 2]
2  아이템 3에 대한 텍스트       [3, 4] 

스택된 임베딩:
 [[1 2]
 [2 2]
 [3 4]]
코사인 유사도 행렬:
           아이템 1     아이템 2     아이템 3
아이템 1  1.000000  0.948683  0.983870
아이템 2  0.948683  1.000000  0.989949
아이템 3  0.983870  0.989949  1.000000 

아이템 1과 아이템 2 사이의 코사인 유사도: 0.9486832980505138
유사도 행렬에서 아이템 1과 아이템 2 사이의 값: 0.9486832980505137


## 같은 방식으로 코사인 유사도 구해보기

In [ ]:
similarity_matrix = cosine_similarity(np.stack(df['np_embedding'].values))
print(len(df))

similarity_matrix


87


array([[1.        , 0.22524766, 0.3382293 , ..., 0.27758975, 0.19772989,
        0.18912892],
       [0.22524766, 1.        , 0.17763758, ..., 0.20024572, 0.20048455,
        0.2933359 ],
       [0.3382293 , 0.17763758, 1.        , ..., 0.3972885 , 0.36964726,
        0.13950558],
       ...,
       [0.27758975, 0.20024572, 0.3972885 , ..., 1.        , 0.43353109,
        0.149592  ],
       [0.19772989, 0.20048455, 0.36964726, ..., 0.43353109, 1.        ,
        0.18109518],
       [0.18912892, 0.2933359 , 0.13950558, ..., 0.149592  , 0.18109518,
        1.        ]])

### matrix를 이용해서, 유사한 기사 행은 묶어서 같은 그룹으로 표시하기




In [ ]:
## 만약 첫번 행에 대해서 각 기사제목과의 유사도를 보고 싶은경우 아래와 같이 similarity_matrix[0]을 하면 숫자가 쭉 나옴
## 이 숫자가 첫번째 행과 n번쨰 행간의 유사도를 의미함 (87개의 숫자)

similarity_matrix[0]

array([1.        , 0.22524766, 0.3382293 , 0.13517586, 0.2705309 ,
       0.39019327, 0.30047607, 0.18854412, 0.27728465, 0.3239338 ,
       0.33098247, 0.22527315, 0.24454082, 0.36924272, 0.22274757,
       0.26673935, 0.28485826, 0.16327613, 0.23113653, 0.21017926,
       0.15138519, 0.1691729 , 0.27860269, 0.29192282, 0.24046992,
       0.27623411, 0.223779  , 0.29256719, 0.21417666, 0.17385915,
       0.25240703, 0.21153296, 0.2835603 , 0.31487127, 0.32843558,
       0.13080636, 0.2107033 , 0.1776104 , 0.29405341, 0.21545171,
       0.27020292, 0.29951508, 0.22408388, 0.29793107, 0.27554851,
       0.22550357, 0.31199684, 0.18667046, 0.16703816, 0.23363679,
       0.25734558, 0.2742323 , 0.2355762 , 0.47732579, 0.0947964 ,
       0.32907903, 0.28596434, 0.23186284, 0.22739075, 0.20626741,
       0.3059628 , 0.29566174, 0.20178707, 0.50801016, 0.19360474,
       0.20729573, 0.18454625, 0.20840404, 0.16005521, 0.18467725,
       0.20304212, 0.27650976, 0.19117676, 0.21955085, 0.26200

In [ ]:
def group_vectors_by_similarity(similarity_matrix, threshold):
    groups = []
    for i in range(similarity_matrix.shape[0]):
        for j in range(i + 1, similarity_matrix.shape[1]):
            if similarity_matrix[i, j] > threshold:
                found_group = False
                for group in groups:
                    if i in group or j in group:
                        group.update([i, j])
                        found_group = True
                        break
                if not found_group:
                    groups.append({i, j})
    return groups

def add_group_info_to_dataframe(df, similarity_matrix, threshold):
    groups = group_vectors_by_similarity(similarity_matrix, threshold)
    print("groups",groups)


    # 일단 모든 행에 group 레이블을 -1으로 선언
    group_labels = -1 * np.ones(len(df), dtype=int)

    # 각 그룹에 속한 행에 그룹번호를 입력하는 과정
    for label, group in enumerate(groups):
        for index in group:
            group_labels[index] = label
    print(group_labels)
    # Add the group labels as a new column to the DataFrame
    df['group_label'] = group_labels
    sorted_df = df.sort_values(by='group_label', ascending=False)

    return sorted_df


df_with_groups = add_group_info_to_dataframe(df, similarity_matrix, 0.95)
df_with_groups

groups [{18, 52}, {84, 38, 71}]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1]


,제목,원문링크,네이버뉴스링크,상세내용,발행일자,np_embedding,group_label
84,내년 공동주택 입주예정물량 27.5만가구…올해보다 24%↓,https://www.yna.co.kr/view/AKR2024020815540000...,https://n.news.naver.com/mnews/article/001/001...,내년 입주예정 물량을 지역별로 보면 <b>서울</b>은 3만1천265가구로 올해(2...,2024-02-08 17:13:52,"[0.02662226, 0.04503617, -0.009569089, 0.03300...",1
38,내년 공동주택 입주예정물량 27만5천가구…올해보다 24% 줄어,https://www.tfmedia.co.kr/news/article.html?no...,https://www.tfmedia.co.kr/news/article.html?no...,"<b>서울</b> 3만1,265가구로 올해보다 9.4% ↑, 경기 7만4,092가구...",2024-02-09 08:20:52,"[0.037945703, 0.03911022, 0.0067509077, 0.0322...",1
71,내년 공동주택 입주예정물량 27.5만세대…올해보다 24%↓,https://www.getnews.co.kr/news/articleView.htm...,https://www.getnews.co.kr/news/articleView.htm...,내년 입주예정 물량을 지역별로 살펴보면 <b>서울</b>은 3만1265세대로 올해(...,2024-02-08 20:02:52,"[0.029231865, 0.04314145, -0.0017631487, 0.031...",1
18,"'불패 상징' 강남3구, 최근 3년보다 다섯 배 많은 아파트 올해 분양",https://www.getnews.co.kr/news/articleView.htm...,https://www.getnews.co.kr/news/articleView.htm...,"9일 부동산R114에 따르면 올해 강남과 서초, 송파 강남 3구 <b>분양</b> ...",2024-02-09 10:12:52,"[-0.0036155821, 0.020978952, 0.061736424, 0.03...",0
52,"'불패 상징' 강남3구, 지난 3년보다 5배 많은 아파트 올해 분양",https://www.yna.co.kr/view/AKR2024020815430000...,https://n.news.naver.com/mnews/article/001/001...,9일 부동산R114에 따르면 올해 강남 3구(강남·서초·송파) <b>분양</b> 물...,2024-02-09 07:00:52,"[-0.008226069, 0.024539953, 0.058757633, 0.032...",0
...,...,...,...,...,...,...,...
28,청약홈 개편·총선 피해 이례적 ‘물량 공세’… 시장 훈풍 기대감,http://www.segye.com/newsView/20240208509585?O...,https://n.news.naver.com/mnews/article/022/000...,최근 교통 호재로 주목받는 김포에서도 ‘김포북변우미린파크리브(1200가구)’가 <b...,2024-02-09 09:04:52,"[0.04200803, 0.015017044, 0.04169931, 0.033121...",-1
27,올해 전국 공동주택 36만4418가구 '입주 예정',http://www.shinailbo.co.kr/news/articleView.ht...,http://www.shinailbo.co.kr/news/articleView.ht...,광역지자체별로는 경기가 11만6595가구로 가장 많았고 인천과 <b>서울</b> 등...,2024-02-09 09:04:52,"[0.008855566, 0.011357863, 0.028823784, 0.0106...",-1
26,'실거주 의무 완화' 논의 평행선…연휴 이후 여야 합의도 미지수,https://news.einfomax.co.kr/news/articleView.h...,https://news.einfomax.co.kr/news/articleView.h...,<b>분양</b>가 상한제 아파트의 실거주 의무를 완화하기 위한 여야 간 논의가 진...,2024-02-09 09:06:52,"[0.03198952, 0.08505149, -0.00084258104, 0.019...",-1
25,"수원시, 작년 경기도 아파트 매매거래 1위…상급지 영통구에 '영통자이 센트럴...",https://www.ksilbo.co.kr/news/articleView.html...,https://www.ksilbo.co.kr/news/articleView.html...,또한 원자잿값 상승으로 인한 착공물량 감소와 이에 따른 <b>분양</b>가 상승 기...,2024-02-09 09:12:52,"[0.010225058, -0.04616904, 0.0018390593, 0.033...",-1


In [ ]:
df_with_groups = add_group_info_to_dataframe(df, similarity_matrix, 0.75)
df_with_groups


groups [{56, 6}, {16, 18, 52, 15}, {38, 71, 74, 80, 84}]
[-1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1  1  1 -1  1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  2 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1  1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  2
 -1 -1  2 -1 -1 -1 -1 -1  2 -1 -1 -1  2 -1 -1]


,제목,원문링크,네이버뉴스링크,상세내용,발행일자,np_embedding,group_label
84,내년 공동주택 입주예정물량 27.5만가구…올해보다 24%↓,https://www.yna.co.kr/view/AKR2024020815540000...,https://n.news.naver.com/mnews/article/001/001...,내년 입주예정 물량을 지역별로 보면 <b>서울</b>은 3만1천265가구로 올해(2...,2024-02-08 17:13:52,"[0.02662226, 0.04503617, -0.009569089, 0.03300...",2
74,내년 입주 예정 아파트 27.5만 가구...올해보다 24% 줄어,http://mbn.mk.co.kr/pages/news/newsView.php?ca...,https://n.news.naver.com/mnews/article/057/000...,"<b>서울</b>은 3만 1천 가구로 올해보다 9.4% 늘어날 전망인 가운데, 경기...",2024-02-08 19:13:52,"[8.4231e-05, 0.0026136732, -0.007939873, 0.024...",2
71,내년 공동주택 입주예정물량 27.5만세대…올해보다 24%↓,https://www.getnews.co.kr/news/articleView.htm...,https://www.getnews.co.kr/news/articleView.htm...,내년 입주예정 물량을 지역별로 살펴보면 <b>서울</b>은 3만1265세대로 올해(...,2024-02-08 20:02:52,"[0.029231865, 0.04314145, -0.0017631487, 0.031...",2
38,내년 공동주택 입주예정물량 27만5천가구…올해보다 24% 줄어,https://www.tfmedia.co.kr/news/article.html?no...,https://www.tfmedia.co.kr/news/article.html?no...,"<b>서울</b> 3만1,265가구로 올해보다 9.4% ↑, 경기 7만4,092가구...",2024-02-09 08:20:52,"[0.037945703, 0.03911022, 0.0067509077, 0.0322...",2
80,내년까지 공동주택 입주예정물량 64만 호,https://news.bbsi.co.kr/news/articleView.html?...,https://news.bbsi.co.kr/news/articleView.html?...,"경기도가 올해 11만 6천 595만 호, 내년 7만 4천 92호 등 19만 687호...",2024-02-08 17:24:52,"[0.043018248, 0.05234624, 0.02781552, 0.023920...",2
...,...,...,...,...,...,...,...
30,금리 정점에 전통 투자 상품 상가로 눈 돌리는 투자자 '코벤트워크 검단 1차'...,https://www.ksilbo.co.kr/news/articleView.html...,https://www.ksilbo.co.kr/news/articleView.html...,단지내상가 <b>분양</b> 소식에 투자자들의 관심이 몰리고 있다. 가장 눈에 띄는...,2024-02-09 09:02:52,"[0.005116945, -0.0025808294, 0.00036155368, 0....",-1
29,"지난해 주택 분양 실적, 13년만에 ‘최저치’",https://news.mtn.co.kr/news-detail/20240209004...,https://news.mtn.co.kr/news-detail/20240209004...,권대중 서강대 일반대학원 부동산학과 교수는 “<b>서울</b>의 신규 <b>분양</...,2024-02-09 09:02:52,"[0.035215043, 0.03273511, 0.03092476, 0.035661...",-1
28,청약홈 개편·총선 피해 이례적 ‘물량 공세’… 시장 훈풍 기대감,http://www.segye.com/newsView/20240208509585?O...,https://n.news.naver.com/mnews/article/022/000...,최근 교통 호재로 주목받는 김포에서도 ‘김포북변우미린파크리브(1200가구)’가 <b...,2024-02-09 09:04:52,"[0.04200803, 0.015017044, 0.04169931, 0.033121...",-1
27,올해 전국 공동주택 36만4418가구 '입주 예정',http://www.shinailbo.co.kr/news/articleView.ht...,http://www.shinailbo.co.kr/news/articleView.ht...,광역지자체별로는 경기가 11만6595가구로 가장 많았고 인천과 <b>서울</b> 등...,2024-02-09 09:04:52,"[0.008855566, 0.011357863, 0.028823784, 0.0106...",-1


In [ ]:
df.to_excel("result.xlsx")
files.download('result.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>